# Demilitarization Degree Prediction - Data Cleaning and Modeling
## Nancy Hamdan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<a id="getting-data"></a>
# Getting the Data

In [ ]:
xls = pd.ExcelFile('AllStatesAndTerritoriesQTR4FY21.xlsx', engine='openpyxl')

In [ ]:
data = pd.DataFrame()

In [ ]:
for name in xls.sheet_names:
    data = pd.concat([data, xls.parse(name)])

In [ ]:
data.head()

,State,Agency Name,NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date,Station Type
0,AL,ABBEVILLE POLICE DEPT,2540-01-565-4700,BALLISTIC BLANKET KIT,10,Kit,15871.59,D,1.0,2018-01-30,State
1,AL,ABBEVILLE POLICE DEPT,1240-DS-OPT-SIGH,OPTICAL SIGHTING AND RANGING EQUIPMENT,1,Each,245.88,D,NaN,2016-06-02,State
2,AL,ABBEVILLE POLICE DEPT,2355-01-553-4634,MINE RESISTANT VEHICLE,1,Each,658000.00,C,1.0,2016-11-09,State
3,AL,ABBEVILLE POLICE DEPT,1240-01-411-1265,"SIGHT,REFLEX",9,Each,333.00,D,1.0,2016-09-14,State
4,AL,ABBEVILLE POLICE DEPT,5855-01-577-7174,"ILLUMINATOR,INFRARED",10,Each,926.00,D,1.0,2017-03-28,State


<a id="data-cleaning"></a>
# Data Cleaning and Feature Engineering

In [ ]:
data_clean = data.copy()

<a id="fixing-data-types"></a>
## Fixing Data Types

In [ ]:
data_clean['DEMIL IC'] = data_clean['DEMIL IC'].astype(str)

<a id="handling-missing-values"></a>
## Handling Missing Values

In [ ]:
data_clean.loc[data_clean['DEMIL IC'] == 'nan', 'DEMIL IC'] = 'Not Reviewed/No Integrity'

In [ ]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130958 entries, 0 to 398
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   State              130958 non-null  object        
 1   Agency Name        130958 non-null  object        
 2   NSN                130958 non-null  object        
 3   Item Name          130958 non-null  object        
 4   Quantity           130958 non-null  int64         
 5   UI                 130958 non-null  object        
 6   Acquisition Value  130958 non-null  float64       
 7   DEMIL Code         130958 non-null  object        
 8   DEMIL IC           130958 non-null  object        
 9   Ship Date          130958 non-null  datetime64[ns]
 10  Station Type       130958 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 12.0+ MB


<a id="removing-erronous-zeros"></a>
## Removing Erroneous Zeros

In [ ]:
err_acquisition_value = data_clean[data_clean['Acquisition Value'] == 0 ]
len(err_acquisition_value)

46

In [ ]:
err_quantity = data_clean[data_clean['Quantity'] == 0 ]
len(err_quantity)

1

In [ ]:
data_clean = data_clean.drop(pd.concat([err_acquisition_value.index.to_series(), err_quantity.index.to_series()]))

<a id="deriving-features"></a>
## Deriving Features

In [ ]:
data_clean['ship_day'] = pd.DatetimeIndex(data_clean['Ship Date']).day
data_clean['ship_month'] = pd.DatetimeIndex(data_clean['Ship Date']).month
data_clean['ship_year'] = pd.DatetimeIndex(data_clean['Ship Date']).year

In [ ]:
data_clean.head()

,State,Agency Name,NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date,Station Type,ship_day,ship_month,ship_year
0,AL,ABBEVILLE POLICE DEPT,2540-01-565-4700,BALLISTIC BLANKET KIT,10,Kit,15871.59,D,1.0,2018-01-30,State,30,1,2018
1,AL,ABBEVILLE POLICE DEPT,1240-DS-OPT-SIGH,OPTICAL SIGHTING AND RANGING EQUIPMENT,1,Each,245.88,D,Not Reviewed/No Integrity,2016-06-02,State,2,6,2016
2,AL,ABBEVILLE POLICE DEPT,2355-01-553-4634,MINE RESISTANT VEHICLE,1,Each,658000.00,C,1.0,2016-11-09,State,9,11,2016
3,AL,ABBEVILLE POLICE DEPT,1240-01-411-1265,"SIGHT,REFLEX",9,Each,333.00,D,1.0,2016-09-14,State,14,9,2016
4,AL,ABBEVILLE POLICE DEPT,5855-01-577-7174,"ILLUMINATOR,INFRARED",10,Each,926.00,D,1.0,2017-03-28,State,28,3,2017


<a id="removing-cols"></a>
## Removing Unnecessary Columns

In [ ]:
len(data_clean[data_clean.UI == 'Each'])/len(data_clean)

0.9616427174298079

In [ ]:
features_todrop = ['Agency Name', 'NSN', 'UI', 'Station Type', 'Ship Date']

In [ ]:
data_clean = data_clean.drop(features_todrop, axis=1)

In [ ]:
data_clean.head()

,State,Item Name,Quantity,Acquisition Value,DEMIL Code,DEMIL IC,ship_day,ship_month,ship_year
0,AL,BALLISTIC BLANKET KIT,10,15871.59,D,1.0,30,1,2018
1,AL,OPTICAL SIGHTING AND RANGING EQUIPMENT,1,245.88,D,Not Reviewed/No Integrity,2,6,2016
2,AL,MINE RESISTANT VEHICLE,1,658000.00,C,1.0,9,11,2016
3,AL,"SIGHT,REFLEX",9,333.00,D,1.0,14,9,2016
4,AL,"ILLUMINATOR,INFRARED",10,926.00,D,1.0,28,3,2017


<a id="features-transformations"></a>
## Features Transformations

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = data_clean.drop('DEMIL Code', axis=1)
y = data_clean['DEMIL Code']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [ ]:
categorical_features = ['State', 'Item Name', 'DEMIL IC']
numerical_features = ['Quantity', 'Acquisition Value', 'ship_day', 'ship_month', 'ship_year']

### Numerical Features

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(x_train[numerical_features])

MinMaxScaler()

In [ ]:
x_train[numerical_features] = scaler.transform(x_train[numerical_features])
x_val[numerical_features] = scaler.transform(x_val[numerical_features])
x_test[numerical_features] = scaler.transform(x_test[numerical_features])

### Categorical Features

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(x_train)

OneHotEncoder(handle_unknown='ignore')

In [ ]:
x_train_enc = ohe.transform(x_train)
x_val_enc = ohe.transform(x_val)
x_test_enc = ohe.transform(x_test)

### Encoding Target

In [ ]:
le = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [ ]:
le.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'Q'], dtype=object)

In [ ]:
y_train_enc = le.transform(y_train)
y_val_enc = le.transform(y_val)
y_test_enc = le.transform(y_test)

<a id="building-models"></a>
# Building Models

In [ ]:
np.random.seed(1)

<a id="baseline-model"></a>
## Building a Baseline

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
forest = RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
forest.fit(x_train_enc, y_train_enc)

RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
forest_y_pred = forest.predict(x_test_enc)

In [ ]:
print(classification_report(y_test_enc, forest_y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3642
           1       0.71      0.30      0.43        33
           2       0.97      0.89      0.93      1386
           3       0.99      0.99      0.99     18529
           4       1.00      0.36      0.53        22
           5       1.00      0.97      0.99      1229
           6       0.99      0.98      0.99      1131

    accuracy                           0.99     25972
   macro avg       0.95      0.79      0.83     25972
weighted avg       0.99      0.99      0.99     25972



* The random forest performed very well in terms of accuracy and precision across the classes. However, the recall of the underrepresented classes (1 and 4) is considerably lower than that of all other classes. To be able to justify using neural network models, the NN models need to perform better than this random forest model as a random forest is less computationally demanding and more easily interpretable.

<a id="building-anns"></a>
## Building ANNs

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.utils import plot_model

In [ ]:
batch = int(len(data_clean)*0.01)

<a id="ann-onehot"></a>
### ANN Using One-Hot Encoding

In [ ]:
input = Input(shape=(x_train_enc.shape[1],))
output = Dense(7, activation='softmax')(input)
model = Model(inputs=input, outputs=output)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train_enc, y_train_enc, epochs=10, batch_size=batch, verbose=2, validation_data=(x_val_enc, y_val_enc))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 7), dtype=float32), dense_shape=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


69/69 - 2s - loss: 1.6104 - accuracy: 0.7077 - val_loss: 1.2925 - val_accuracy: 0.7397 - 2s/epoch - 27ms/step
Epoch 2/10
69/69 - 1s - loss: 1.0851 - accuracy: 0.7462 - val_loss: 0.9081 - val_accuracy: 0.7643 - 856ms/epoch - 12ms/step
Epoch 3/10
69/69 - 1s - loss: 0.7984 - accuracy: 0.7843 - val_loss: 0.7049 - val_accuracy: 0.8049 - 898ms/epoch - 13ms/step
Epoch 4/10
69/69 - 1s - loss: 0.6387 - accuracy: 0.8190 - val_loss: 0.5829 - val_accuracy: 0.8313 - 865ms/epoch - 13ms/step
Epoch 5/10
69/69 - 1s - loss: 0.5354 - accuracy: 0.8430 - val_loss: 0.4983 - val_accuracy: 0.8523 - 868ms/epoch - 13ms/step
Epoch 6/10
69/69 - 1s - loss: 0.4602 - accuracy: 0.8648 - val_loss: 0.4345 - val_accuracy: 0.8723 - 868ms/epoch - 13ms/step
Epoch 7/10
69/69 - 1s - loss: 0.4019 - accuracy: 0.8852 - val_loss: 0.3839 - val_accuracy: 0.8918 - 867ms/epoch - 13ms/step
Epoch 8/10
69/69 - 1s - loss: 0.3550 - accuracy: 0.9071 - val_loss: 0.3427 - val_accuracy: 0.9132 - 872ms/epoch - 13ms/step
Epoch 9/10
69/69 - 1s 

In [ ]:
y_pred_proba = model.predict(x_test_enc)
y_pred = np.argmax(y_pred_proba, axis=1)

In [ ]:
print(classification_report(y_test_enc, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3642
           1       0.00      0.00      0.00        33
           2       0.99      0.62      0.76      1386
           3       0.94      1.00      0.97     18529
           4       0.00      0.00      0.00        22
           5       0.99      0.71      0.83      1229
           6       1.00      0.74      0.85      1131

    accuracy                           0.95     25972
   macro avg       0.70      0.58      0.62     25972
weighted avg       0.95      0.95      0.94     25972



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


* The ANN using one-hot encoding performed well in terms of accuracy and precision across the classes besides classes 1 and 4 which it could not predict at all. This model performed worse than the benchmark model.

<a id="ann-embeddings"></a>
### ANN Using Feature Embedding

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import StringLookup
from tensorflow.keras import layers
import math

In [ ]:
# an input layer is created for each feature, it has to be a dictionary because 
# data is passed as a dictionary and this way it will be mapped to the right input layer
inputs, encoded_features = {}, [] 
for col in x_train.columns:
  if col in categorical_features:
    vocab = list(data_clean[col].unique()) # getting vocobulary of the feature

    # this is a lookup that converts strings to their right integer indices
    lookup = StringLookup(
                vocabulary=vocab,
                mask_token=None,
                num_oov_indices=0,
                output_mode="int",
            )
    
    feature_input = layers.Input(name=col, shape=(), dtype=tf.string)
    encoded_feature = lookup(feature_input) # getting the integer index of the string input
    
    embedding_dims = int(math.sqrt(len(vocab)))
    embedding = layers.Embedding(input_dim=len(vocab), output_dim=embedding_dims) # creates embedding layer for the feature

    encoded_feature = embedding(encoded_feature) # use the embedding layer on the feature, converts index value of the feature to embedding representation
    
    encoded_features.append(encoded_feature)
    inputs[col] = feature_input
  else:
    # if feature is numeric then use as-is
    feature_input = layers.Input(name=col, shape=(), dtype=tf.float32)
    # reshaping because all layers' outputs have to have the same shape and this is the shape the lookup layer outputs 
    encoded_feature = tf.expand_dims(feature_input, -1)
    
    encoded_features.append(encoded_feature)
    inputs[col] = feature_input

In [ ]:
all_features = layers.concatenate(encoded_features)
output = Dense(7, activation='softmax')(all_features)
model = Model(inputs=inputs, outputs=output)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(dict(x_train), y_train_enc, epochs=10, batch_size=batch, verbose=2, validation_data=(dict(x_val), y_val_enc))

Epoch 1/10
69/69 - 2s - loss: 1.5941 - accuracy: 0.6040 - val_loss: 1.0376 - val_accuracy: 0.9182 - 2s/epoch - 23ms/step
Epoch 2/10
69/69 - 1s - loss: 0.6719 - accuracy: 0.9338 - val_loss: 0.4409 - val_accuracy: 0.9349 - 555ms/epoch - 8ms/step
Epoch 3/10
69/69 - 1s - loss: 0.3400 - accuracy: 0.9463 - val_loss: 0.2772 - val_accuracy: 0.9439 - 559ms/epoch - 8ms/step
Epoch 4/10
69/69 - 1s - loss: 0.2265 - accuracy: 0.9545 - val_loss: 0.2114 - val_accuracy: 0.9498 - 602ms/epoch - 9ms/step
Epoch 5/10
69/69 - 1s - loss: 0.1744 - accuracy: 0.9607 - val_loss: 0.1782 - val_accuracy: 0.9517 - 594ms/epoch - 9ms/step
Epoch 6/10
69/69 - 1s - loss: 0.1453 - accuracy: 0.9624 - val_loss: 0.1585 - val_accuracy: 0.9537 - 575ms/epoch - 8ms/step
Epoch 7/10
69/69 - 1s - loss: 0.1271 - accuracy: 0.9639 - val_loss: 0.1455 - val_accuracy: 0.9560 - 590ms/epoch - 9ms/step
Epoch 8/10
69/69 - 1s - loss: 0.1149 - accuracy: 0.9649 - val_loss: 0.1362 - val_accuracy: 0.9603 - 590ms/epoch - 9ms/step
Epoch 9/10
69/69 -

In [ ]:
y_pred_proba = model.predict(dict(x_test))
y_pred = np.argmax(y_pred_proba, axis=1)

In [ ]:
print(classification_report(y_test_enc, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      3642
           1       1.00      0.36      0.53        33
           2       0.81      0.82      0.82      1386
           3       0.98      0.98      0.98     18529
           4       1.00      0.36      0.53        22
           5       0.88      0.86      0.87      1229
           6       0.97      0.89      0.93      1131

    accuracy                           0.96     25972
   macro avg       0.94      0.75      0.80     25972
weighted avg       0.96      0.96      0.96     25972



* The ANN using embeddings performed well in terms of accuracy and precision across the classes. This ANN has also produced way higher recall scores across the classes besides class 1 and 4 for which it had the same recall scores as the benchmark. This model has outperformed the benchmark model in producing higher recall scores for most classes.

<a id="dnn-onhot-embedding"></a>
## Building a DNN

### DNN Using Both One-Hot Encoding and Embedding

In [ ]:
# an input layer is created for each feature, has to be a dictionary because 
# data is passed as a dictionary and this way it will be mapped to the right input layer
def create_inputs():
  inputs = {}
  for col in x_train.columns:
    if col in categorical_features:
      inputs[col] = layers.Input(name=col, shape=(), dtype=tf.string)
    else:
      inputs[col] = layers.Input(name=col, shape=(), dtype=tf.float32)
  return inputs

In [ ]:
def encode_inputs(inputs, use_embedding):
  encoded_features = []

  for col in x_train.columns:
    if col in categorical_features:
      vocab = list(data_clean[col].unique()) # getting vocobulary of the feature

      # this is a lookup that converts strings to their right integer indices, binary means return one-hot representation
      lookup = StringLookup(
                  vocabulary=vocab,
                  mask_token=None,
                  num_oov_indices=0,
                  output_mode="int" if use_embedding else "binary"
              )
      
      if use_embedding:
        encoded_feature = lookup(inputs[col]) # getting the integer index of the string input
        embedding_dims = int(math.sqrt(len(vocab)))
        embedding = layers.Embedding(input_dim=len(vocab), output_dim=embedding_dims) # creates embedding layer for the feature
        encoded_feature = embedding(encoded_feature) # use the layer on the feature, converts index value of the feature to embedding representation
      
      else:
        encoded_feature = lookup(tf.expand_dims(inputs[col], -1)) # get one-hot encoding of feature
      
      encoded_features.append(encoded_feature)
    else:
      # if feature is numeric then use as-is
      # reshaping because all layers' outputs have to have the same shape and this is the shape the lookup layer outputs
      encoded_feature = tf.expand_dims(inputs[col], -1)
      encoded_features.append(encoded_feature)

  all_features = layers.concatenate(encoded_features)

  return all_features

In [ ]:
inputs = create_inputs()
one_hot_features = encode_inputs(inputs, False)
embedded_features = encode_inputs(inputs, True)

In [ ]:
wide = Dense(20, activation='relu')(one_hot_features)
wide = Dense(10, activation='relu')(wide)
wide = Dense(20, activation='relu')(wide)
wide = layers.Dropout(0.1)(wide)

deep = Dense(20, activation='relu')(embedded_features)
deep = Dense(10, activation='relu')(deep)
deep = Dense(20, activation='relu')(deep)
deep = layers.Dropout(0.1)(deep)

merged = layers.concatenate([wide, deep])
output = Dense(units=7, activation="softmax")(merged)

model = Model(inputs=inputs, outputs=output)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(dict(x_train), y_train_enc, epochs=20, batch_size=batch, verbose=2, validation_data=(dict(x_val), y_val_enc))

Epoch 1/20
69/69 - 5s - loss: 1.3861 - accuracy: 0.6559 - val_loss: 0.6888 - val_accuracy: 0.7166 - 5s/epoch - 69ms/step
Epoch 2/20
69/69 - 3s - loss: 0.4593 - accuracy: 0.8285 - val_loss: 0.2130 - val_accuracy: 0.9545 - 3s/epoch - 46ms/step
Epoch 3/20
69/69 - 3s - loss: 0.1441 - accuracy: 0.9621 - val_loss: 0.1162 - val_accuracy: 0.9644 - 3s/epoch - 45ms/step
Epoch 4/20
69/69 - 3s - loss: 0.0990 - accuracy: 0.9677 - val_loss: 0.1018 - val_accuracy: 0.9660 - 3s/epoch - 47ms/step
Epoch 5/20
69/69 - 3s - loss: 0.0861 - accuracy: 0.9702 - val_loss: 0.0948 - val_accuracy: 0.9695 - 3s/epoch - 47ms/step
Epoch 6/20
69/69 - 3s - loss: 0.0805 - accuracy: 0.9716 - val_loss: 0.0912 - val_accuracy: 0.9682 - 3s/epoch - 47ms/step
Epoch 7/20
69/69 - 3s - loss: 0.0756 - accuracy: 0.9730 - val_loss: 0.0890 - val_accuracy: 0.9701 - 3s/epoch - 47ms/step
Epoch 8/20
69/69 - 3s - loss: 0.0722 - accuracy: 0.9734 - val_loss: 0.0867 - val_accuracy: 0.9694 - 3s/epoch - 47ms/step
Epoch 9/20
69/69 - 3s - loss: 0.

In [ ]:
y_pred_proba = model.predict(dict(x_test))
y_pred = np.argmax(y_pred_proba, axis=1)
print(classification_report(y_test_enc, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3642
           1       1.00      0.61      0.75        33
           2       0.89      0.85      0.87      1386
           3       0.99      0.99      0.99     18529
           4       0.92      0.50      0.65        22
           5       0.89      0.92      0.91      1229
           6       0.99      0.99      0.99      1131

    accuracy                           0.98     25972
   macro avg       0.95      0.83      0.88     25972
weighted avg       0.98      0.98      0.98     25972



* The DNN model which uses both one-hot encoding and embeddings has outperformed all models. It has produced high precision, recall and accuracy scores. Its precision across the classes is only slightly worse than that of the benchmark and is higher than that of other ANNs. Its recall scores are the best recorded so far.

<a id="dnn-hyperparam-tuning"></a>
### Hyperparameter Tuning

In [ ]:
data_clean['DEMIL Code'].value_counts()/len(data_clean)*100

D    71.364875
A    13.992207
C     5.359701
F     4.727472
Q     4.326264
B     0.119361
E     0.110120
Name: DEMIL Code, dtype: float64

In [ ]:
le.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'Q'], dtype=object)

In [ ]:
class_weight = {0:(1/13.983109) , 1:(1/0.119122), 2:(1/5.354388), 3:(1/71.380137), 4:(1/0.109959), 5:(1/4.732815), 6:(1/4.320469)}

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(dict(x_train), y_train_enc, epochs=20, batch_size=batch, verbose=2, validation_data=(dict(x_val), y_val_enc), class_weight=class_weight)

Epoch 1/20
69/69 - 5s - loss: 0.0140 - accuracy: 0.9650 - val_loss: 0.1376 - val_accuracy: 0.9486 - 5s/epoch - 70ms/step
Epoch 2/20
69/69 - 3s - loss: 0.0083 - accuracy: 0.9591 - val_loss: 0.1426 - val_accuracy: 0.9501 - 3s/epoch - 47ms/step
Epoch 3/20
69/69 - 3s - loss: 0.0076 - accuracy: 0.9590 - val_loss: 0.1463 - val_accuracy: 0.9477 - 3s/epoch - 47ms/step
Epoch 4/20
69/69 - 3s - loss: 0.0066 - accuracy: 0.9591 - val_loss: 0.1352 - val_accuracy: 0.9521 - 3s/epoch - 46ms/step
Epoch 5/20
69/69 - 3s - loss: 0.0063 - accuracy: 0.9603 - val_loss: 0.1340 - val_accuracy: 0.9519 - 3s/epoch - 46ms/step
Epoch 6/20
69/69 - 3s - loss: 0.0059 - accuracy: 0.9617 - val_loss: 0.1393 - val_accuracy: 0.9527 - 3s/epoch - 46ms/step
Epoch 7/20
69/69 - 3s - loss: 0.0059 - accuracy: 0.9618 - val_loss: 0.1571 - val_accuracy: 0.9447 - 3s/epoch - 46ms/step
Epoch 8/20
69/69 - 3s - loss: 0.0053 - accuracy: 0.9612 - val_loss: 0.1427 - val_accuracy: 0.9498 - 3s/epoch - 47ms/step
Epoch 9/20
69/69 - 3s - loss: 0.

In [ ]:
y_pred_proba = model.predict(dict(x_test))
y_pred = np.argmax(y_pred_proba, axis=1)
print(classification_report(y_test_enc, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3642
           1       0.16      0.88      0.27        33
           2       0.74      0.91      0.82      1386
           3       1.00      0.96      0.98     18529
           4       0.28      0.64      0.39        22
           5       0.84      0.98      0.91      1229
           6       0.99      0.98      0.99      1131

    accuracy                           0.96     25972
   macro avg       0.71      0.90      0.76     25972
weighted avg       0.97      0.96      0.96     25972



* Using the custom class weights has significantly improved the DNN's recall scores for the underrepresented classes 1 and 4 to be like those of the other classes. That came at the cost of worsening precision.

<a id="conclusion"></a>
# Conclusion

Predicting the DEMIL Code is an output sensitive problem where it is more important to have less false negatives than it is to have less false positives. For example, since DEMIL Code ‘B’ means mutilation to the
point of scrap is required worldwide, falsely predicting that a DEMIL Code is ‘B’ (false positive) is less bad than falsely predicting an actual DEMIL Code ‘B’ as another code (false negative) that could potentially
represent a lower demilitarization degree. Because of that, having higher recall scores is more important and thus the DNN which uses both one-hot encoding and embeddings and custom class weights is the best performing model.